# Strain

Note the **sample radius is 9.5 mm**, where the **PWJ radius is 2.5 mm**. Meaning for the waterjet to be fully on the sample you need an **$x$ position of $\pm$7 mm**. The step size was 0.2 mm ranging from $\pm$ 11.8 mm.

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import pandas as pd
from pathlib import Path
from rich import print

ROOT = Path('.').absolute().parent
# NOTE: If you want the ROOT directory to be the same as the notebook 
# ROOT = Path('.').absolute()
DATA = ROOT / "paraview/experiment_overlap"
DATA_60MM = ROOT / "paraview/experiment_overlap_post_60mm"
# NOTE: If you put the data in the same path as the notebook
#DATA = ROOT 
EXPORT = ROOT / "data"
THESIS = ROOT / "../../thesis/fig/device/fea"

VTR = 21.167 # mm/s

print(ROOT)
print(DATA)
print(DATA_60MM)
print("VTR:", VTR, "mm/s")

In [ ]:
data = list(DATA.glob("*.tsv"))
data = sorted(data)
print(data)

In [ ]:
df = {}

for d, l in zip(data, ['bot', 'mid', 'top']):
    df[l] = pd.read_csv(d, delimiter='\t')
    df[l].rename(columns={"Time": "Time (s)", "avg(strain(z,z))": "Strain"}, inplace=True)
    df[l]["Position"] = [l] * df[l].shape[0]

print(df[l].columns)

In [ ]:
cols = ["Time (s)", "Position", "Strain"]
for k in df.keys():
    print(df[k][cols].head(10))

In [ ]:
df_proc = (pd.concat([df['bot'], df['mid'], df['top']], axis=0)
    .pivot_table("Strain", index="Time (s)", columns=["Position"])
    .agg(["mean", "std"], axis=1))

df_proc

## Corrected data for 60mm post

In [ ]:
block_names = {
    'block=1': 'bot',
    'block=2': 'mid',
    'block=3': 'top',
}

df = pd.read_csv(DATA_60MM / "data.tsv", delimiter='\t+', engine='python')
df.rename(
    columns={
        "Block Name": "Position", 
        "Time": "Time (s)",
        "avg(strain(z,z))": "Strain", 
        "avg(displacement (Magnitude))": "Displacement (mm)"}, 
    inplace=True
)

df['Position'] = df['Position'].apply(lambda x: block_names[x])
df

In [ ]:
df_proc = df.pivot_table(
        index="Time (s)", 
        values="Strain", 
        columns="Position"
).agg(["mean", "std"], axis=1)
df_proc

## Get interesting points

In [ ]:
df_pts = pd.read_feather(ROOT / "data/strain_sim_corrected.feather")
msk_0 = np.abs(df_pts["Position (mm)"]) < 1E-3
msk_max = df_pts["Strain"] == df_pts["Strain"].max()
msk_min = df_pts["Strain"] == df_pts["Strain"].min()
msk_max_mid = np.abs(df_pts["Time (s)"] - 0.10395) < 1E-3
msk_min_mid = np.abs(df_pts["Time (s)"] - 1.00155) < 1E-3
df_interest = df_pts[msk_0 + msk_max + msk_min + msk_max_mid + msk_min_mid]
df_interest.reset_index(inplace=True)
df_interest

In [ ]:
def time2dist(x):
    return (x * VTR) - 11.8 


def dist2time(x):
    return (x + 11.8) / VTR

with plt.style.context('thesis'):
    fig, ax = plt.subplots(1, 1)

    df_proc.plot(y="mean", ax=ax, legend=False)
    

    pwj_sample_edge = 7
    df_proc[
        (dist2time(-pwj_sample_edge) <= df_proc.index) 
        & (df_proc.index <= dist2time(pwj_sample_edge))
    ].plot(y="mean", ax=ax, legend=False, style="C1") 

    ymin = df_proc["mean"].values - df_proc["std"].values
    ymax = df_proc["mean"].values + df_proc["std"].values
    ax.fill_between(df_proc.index, ymin, ymax, alpha=0.3)

    xmin, xmax = ax.get_xlim()
    ax.hlines(0, -xmax, xmax * 2, '#999', linewidth=1.0, linestyle='--')

    y_center = df_proc["mean"][abs(df_proc.index - dist2time(0)) < 5E-3].mean()
    ax.hlines(y_center, -xmax, xmax * 2, "C1", alpha=0.5)
    ax.set_xlim(xmin, xmax)

    ymin, ymax = ax.get_ylim()
    ax.vlines(dist2time(0), -5, 5, "C1", alpha=0.5)
    ax.set_ylim(ymin, ymax)

    rescale = 1E-4
    ticks = np.round(ax.get_yticks() / rescale)
    ax.set_yticklabels(ticks)

    ax.legend(["Full sweep", "Only on sample"], title="Water jet")
    df_interest.plot.scatter(
        x="Time (s)",
        y="Strain",
        s=40,
        c='none',
        edgecolor='#56b4e9',
        legend=False,
        ax=ax,
        zorder=10,
    )
    
    offset = ((-0.06, -0.1E-4),
              (-0.015, -0.6E-4),
              (-0.05, -0.4E-4),
              (-0.015, 0.4E-4),
              (+0.03, -0.1E-4))
    print("Points of interest")
    for i, lbl in enumerate("abcde"):
        x = df_interest.iloc[i]["Time (s)"]
        y = df_interest.iloc[i]["Strain"]
        print(x, y)
        dx, dy = offset[i]
        ax.annotate(lbl, (x + dx, y + dy), color="#56b4e9")

    secax = ax.secondary_xaxis('top', functions=(time2dist, dist2time))

    #ax.set_title("Strain on Al 6061-T6 post at sensor")
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Strain $\\times 10^{{{}}}$".format(int(np.log10(rescale))))
    ax.minorticks_on()
    ax.xaxis.set_tick_params('both', top=False)

    secax.set_xlabel('Position (mm)')
    secax.minorticks_on()
    
    ax.yaxis.set_tick_params('both', right=False)
    
    secyax = ax.secondary_yaxis('right', functions=(strain2volt, volt2strain))
    secyax.set_ylabel('Voltage (V)')
    secyax.minorticks_on()
    
    fig.subplots_adjust(right=0.85)
    
    fig.savefig(THESIS / "strain.svg", bbox_inches="tight")
    #fig.tight_layout()

In [ ]:
P_hydraulic = 30E6 # Pa
# Multiplied by -1 for Circuit compatibility
strain_sim_at_30MPa = 0.000974938
volt_at_30MPa = 1.595

def calc_strain_max(x, v_cc=3.3, v_meas=1.595):
    return x * v_cc / v_meas


strain_max = calc_strain_max(strain_sim_at_30MPa)
print(f"Strain 30 MPa Hydraulic Press: {strain_sim_at_30MPa:0.3g}")
print(f"Strain max: {strain_max:0.3g}")


def strain2volt(x, strain_max=strain_max, v_cc=3.3):
    return x * v_cc / strain_max


def volt2strain(x, strain_max=strain_max, v_cc=3.3):
    return x * strain_max / v_cc

In [ ]:
def time2dist(x):
    return (x * VTR) - 11.8 


def dist2time(x):
    return (x + 11.8) / VTR

fig, ax = plt.subplots(1, 1)

df_export.loc

df_export.plot(y="Sensor Strain", ax=ax, legend=False)

pwj_sample_edge = 7
df_export[
    (dist2time(-pwj_sample_edge) <= df_export.index) 
    & (df_export.index <= dist2time(pwj_sample_edge))
].plot(y="Sensor Strain", ax=ax, legend=False, style="C1") 

ymin = df_export["Sensor Strain"].values - df_export["Sensor Strain std"].values
ymax = df_export["Sensor Strain"].values + df_export["Sensor Strain std"].values
ax.fill_between(df_export.index, ymin, ymax, alpha=0.3)

xmin, xmax = ax.get_xlim()
ax.hlines(0, -xmax, xmax * 2, '#999', linewidth=1.0, linestyle='--')

y_center = df_export["Sensor Strain"][abs(df_export.index - dist2time(0)) < 5E-3].mean()
ax.hlines(y_center, -xmax, xmax * 2, "C1", alpha=0.5)
ax.set_xlim(xmin, xmax)

ymin, ymax = ax.get_ylim()
ax.vlines(dist2time(0), -5, 5, "C1", alpha=0.5)
ax.set_ylim(ymin, ymax)

rescale = 1E-4
ticks = np.round(ax.get_yticks() / rescale)
ax.set_yticklabels(ticks)

ax.legend(["Full sweep", "Sample only"], title="PWJ")

ax.set_title("Strain Gauge")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Strain $\\times 10^{{{}}}$".format(int(np.log10(rescale))))
ax.minorticks_on()

secxax = ax.secondary_xaxis('top', functions=(time2dist, dist2time))
secxax.set_xlabel('Position (mm)')
secxax.minorticks_on()

secyax = ax.secondary_yaxis('right', functions=(strain2volt, volt2strain))
secyax.set_ylabel('Voltage (V)')
secyax.minorticks_on()

fig.tight_layout()

In [ ]:
df_export = df_proc.rename(columns={"mean": "Strain", "std": "Strain std"})
df_export["Sensor Strain"] = df_export["Strain"] * -1
df_export["Sensor Strain std"] = df_export["Strain std"] * -1
df_export["Position (mm)"] = time2dist(df_proc.index)
df_export["Voltage (V)"] = df_export["Sensor Strain"].apply(strain2volt)
df_export.reset_index().to_feather(EXPORT / "strain_sim_corrected.feather")
df_export.head()